In [1]:
import os
memory = '100g'
pyspark_submit_args = '--packages org.postgresql:postgresql:42.2.9 pyspark-shell' + ' --driver-memory ' + memory + ' pyspark-shell ' + '--num-executors 3'
import time

os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('BDA_A2').master('local').getOrCreate()

In [3]:
df1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/assignment1") \
    .option("dbtable", "GHTORRENT_RECORDS") \
    .option('user', 'postgres') \
    .option('password', 'admin') \
    .option('driver', 'org.postgresql.Driver') \
    .load()

df2 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/assignment1") \
    .option("dbtable", "IMPORTANT_REPOS") \
    .option('user', 'postgres') \
    .option('password', 'admin') \
    .option('driver', 'org.postgresql.Driver') \
    .load()

In [4]:
print("Table1")
df1.printSchema()

print("Table2")
df2.printSchema()

Table1
root
 |-- logging_level: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- downloader_id: string (nullable = true)
 |-- retrieval_stage: string (nullable = true)
 |-- request_status: string (nullable = true)
 |-- url: string (nullable = true)
 |-- access_key: string (nullable = true)

Table2
root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- owner_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- language: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- forked_from: string (nullable = true)
 |-- deleted: string (nullable = true)
 |-- updated_at: string (nullable = true)



In [5]:
df1.createOrReplaceTempView('GHTORRENT_RECORDS')
df2.createOrReplaceTempView('IMPORTANT_REPOS')

## Queries

In [6]:
query2 = "SELECT COUNT(*) FROM GHTORRENT_RECORDS"
result2 = spark.sql(query2)
print("How many records does the table contain?") 
result2.show()

How many records does the table contain?
+--------+
|count(1)|
+--------+
| 9669634|
+--------+



In [7]:
query3 = "SELECT COUNT(*) FROM GHTORRENT_RECORDS WHERE LOGGING_LEVEL='WARN'"
result3 = spark.sql(query3)
print("Count the number of WARNing messages.")
result3.show()

Count the number of WARNing messages.
+--------+
|count(1)|
+--------+
|  132158|
+--------+



In [8]:
# 78588
query4 = '''SELECT COUNT(DISTINCT URL) 
               FROM GHTORRENT_RECORDS 
               WHERE RETRIEVAL_STAGE = 'api_client' AND URL != 'NULL' '''
result4 = spark.sql(query4)
print("How many repositories were processed in total?")
result4.show()

How many repositories were processed in total?
+-------------------+
|count(DISTINCT URL)|
+-------------------+
|              78369|
+-------------------+



In [9]:
query5 = """SELECT COUNT(RETRIEVAL_STAGE), DOWNLOADER_ID
            FROM GHTORRENT_RECORDS 
            WHERE RETRIEVAL_STAGE='api_client' AND URL != 'NULL'
            GROUP BY DOWNLOADER_ID
            ORDER BY COUNT(RETRIEVAL_STAGE) DESC 
            LIMIT 10"""
result5 = spark.sql(query5)
result5.show()

+----------------------+-------------+
|count(RETRIEVAL_STAGE)|DOWNLOADER_ID|
+----------------------+-------------+
|                 85528|           13|
|                 19046|            4|
|                 18948|           18|
|                 18926|           10|
|                 18911|           40|
|                 18616|           39|
|                 18614|           38|
|                 18604|           47|
|                 18463|            1|
|                 18452|           24|
+----------------------+-------------+



In [10]:
# [(79623, '13'), (1378, '21'), (1134, '40'), (368, '18'), (357, '42'), (356, '9'), (352, '4'), (342, '25'), (333, '22'), (332, '6')]
query6 = """SELECT COUNT(RETRIEVAL_STAGE), DOWNLOADER_ID
            FROM GHTORRENT_RECORDS 
            WHERE RETRIEVAL_STAGE='api_client' AND REQUEST_STATUS='Failed'
            GROUP BY DOWNLOADER_ID
            ORDER BY COUNT(RETRIEVAL_STAGE) DESC 
            LIMIT 10"""
result6 = spark.sql(query6)
result6.show()

+----------------------+-------------+
|count(RETRIEVAL_STAGE)|DOWNLOADER_ID|
+----------------------+-------------+
|                 79623|           13|
|                  1378|           21|
|                  1134|           40|
|                   368|           18|
|                   357|           42|
|                   356|            9|
|                   352|            4|
|                   342|           25|
|                   333|           22|
|                   332|            6|
+----------------------+-------------+



In [ ]:
# [(2662487, '10')]
# Takes a lot of time to run
query7 = """
            SELECT NEWT.TS
            FROM (SELECT LOGGING_LEVEL, SUBSTRING(TIMESTAMP, 12, 2) AS TS, DOWNLOADER_ID, RETRIEVAL_STAGE, REQUEST_STATUS, URL, ACCESS_KEY
                  FROM GHTORRENT_RECORDS) AS NEWT
            GROUP BY NEWT.TS
            ORDER BY COUNT(NEWT.TS) DESC
            LIMIT 1
            """
result7 = spark.sql(query7)
result7.show()

In [12]:
# [('https://api.github.com/repos/greatfakeman/Tabchi', 79539)]
query8 = '''
        SELECT  URL
        FROM GHTORRENT_RECORDS
        WHERE URL != 'NULL'
        GROUP BY URL
        ORDER BY COUNT(URL) DESC
        LIMIT 1'''
result8 = spark.sql(query8)
result8.show()

+--------------------+
|                 URL|
+--------------------+
|https://api.githu...|
+--------------------+



In [14]:
# [('ac6168f8776', 79623), ('46f11b5791b', 1340), ('9115020fb01', 1134), ('c1240f63b5b', 371), ('2776f3ba0a5', 368)]
query9 = '''
        SELECT  ACCESS_KEY, COUNT(ACCESS_KEY) AS key_count
        FROM GHTORRENT_RECORDS
        WHERE ACCESS_KEY != 'NULL'
        GROUP BY ACCESS_KEY
        ORDER BY key_count DESC
        LIMIT 5'''
result9 = spark.sql(query9)
result9.show()

+-----------+---------+
| ACCESS_KEY|key_count|
+-----------+---------+
|ac6168f8776|    79623|
|46f11b5791b|     1340|
|9115020fb01|     1134|
|c1240f63b5b|      371|
|2776f3ba0a5|      368|
+-----------+---------+



### Table 2 Queries


In [15]:
query10 = "SELECT COUNT(*) FROM IMPORTANT_REPOS"
result10 = spark.sql(query10)
print("How many records does the IMPORTANT_REPOS table contain?")
result10.show()

How many records does the IMPORTANT_REPOS table contain?
+--------+
|count(1)|
+--------+
|    1435|
+--------+



In [16]:
query11 = '''
            SELECT COUNT(ir.URL)
            FROM GHTORRENT_RECORDS gt
            INNER JOIN IMPORTANT_REPOS ir
                ON gt.URL != 'NULL' AND gt.URL=ir.URL
            '''
result11 = spark.sql(query11)
print("How many records in the log file refer to entries in the interesting file?")
result11.show()

How many records in the log file refer to entries in the interesting file?
+----------+
|count(URL)|
+----------+
|    158348|
+----------+



In [18]:
query12 = '''
            SELECT ir.URL, COUNT(ir.URL)
            FROM GHTORRENT_RECORDS gt
            INNER JOIN IMPORTANT_REPOS ir
            ON gt.URL != 'NULL' AND gt.URL=ir.URL AND gt.REQUEST_STATUS = 'Failed' 
            GROUP BY ir.URL
            ORDER BY COUNT(ir.URL) DESC
            LIMIT 5'''
result12 = spark.sql(query12)
print("Which of the interesting repositories has the most failed API calls?")
result12.show()

Which of the interesting repositories has the most failed API calls?
+--------------------+----------+
|                 URL|count(URL)|
+--------------------+----------+
|https://api.githu...|         5|
|https://api.githu...|         3|
|https://api.githu...|         3|
|https://api.githu...|         2|
|https://api.githu...|         2|
+--------------------+----------+



In [17]:
query10 = '''
            SELECT COUNT(DISTINCT URL) 
            FROM GHTORRENT_RECORDS 
            WHERE DOWNLOADER_ID = '22' AND URL != 'NULL'
            '''

start_time = time.time()
spark.sql(query10)
end_time = time.time()

print("TimeTaken: {} ms".format(end_time-start_time))


TimeTaken: 0.24361324310302734 ms


In [6]:
query11 = '''
            SELECT COUNT(DISTINCT URL) 
            FROM GHTORRENT_RECORDS 
            WHERE DOWNLOADER_ID = '22' AND URL != 'NULL'
            '''

start_time = time.time()
spark.sql(query11)
end_time = time.time()

print("TimeTaken: {} ms".format(end_time-start_time))


TimeTaken: 0.3787837028503418 ms
